## ONNX Model Visualization

Print visulization

In [11]:
from pathlib import Path
import onnx


project_root = Path("/workspaces/dtu_mlops_project")
onnx_files = sorted((project_root / "logs/train/runs").glob("*/model.onnx"), key=lambda p: p.stat().st_mtime)
if not onnx_files:
    raise FileNotFoundError(
        f"No ONNX files found in {project_root / 'logs/train/runs'}. Run training with export_onnx=True to generate one."
    )
onnx_path = onnx_files[-1]
print(f"Using {onnx_path}")

model = onnx.load(onnx_path)
onnx.checker.check_model(model)
print(onnx.helper.printable_graph(model.graph))

Using /workspaces/dtu_mlops_project/logs/train/runs/2026-01-20_15-48-19/model.onnx
graph main_graph (
  %input[FLOAT, batch_sizex1x28x28]
) initializers (
  %model.0.weight[FLOAT, 8x1x3x3]
  %model.0.bias[FLOAT, 8]
  %model.3.weight[FLOAT, 16x8x3x3]
  %model.3.bias[FLOAT, 16]
  %classifier.1.weight[FLOAT, 10x784]
  %classifier.1.bias[FLOAT, 10]
) {
  %/model/model.0/Conv_output_0 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%input, %model.0.weight, %model.0.bias)
  %/model/model.1/Relu_output_0 = Relu(%/model/model.0/Conv_output_0)
  %/model/model.2/MaxPool_output_0 = MaxPool[ceil_mode = 0, dilations = [1, 1], kernel_shape = [2, 2], pads = [0, 0, 0, 0], strides = [2, 2]](%/model/model.1/Relu_output_0)
  %/model/model.3/Conv_output_0 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%/model/model.2/MaxPool_output_0, %model.3.weight, %model.3.bias)
  %/model/model.4/Relu_output_0 = Re

/tmp/ipykernel_10041/1814252964.py:15: DeprecationWarning: Deprecated since 1.19. Consider using onnx.printer.to_text() instead.
  print(onnx.helper.printable_graph(model.graph))



Visualize the exported ONNX model graph using Netron.

> uv run netron logs/train/runs/2026-01-20_15-48-19/model.onnx

## ONNX Runtime Inference
Use ONNX Runtime to optimize and run inference on the exported model.

In [12]:
import onnxruntime as rt
import numpy as np

# Create ONNX Runtime inference session
ort_session = rt.InferenceSession(str(onnx_path))

# Get input and output names
input_names = [i.name for i in ort_session.get_inputs()]
output_names = [i.name for i in ort_session.get_outputs()]

print(f"Input names: {input_names}")
print(f"Output names: {output_names}")

# Get input shape
input_shape = ort_session.get_inputs()[0].shape
print(f"Input shape: {input_shape}")

Input names: ['input']
Output names: ['output']
Input shape: ['batch_size', 1, 28, 28]


In [13]:
# Create random input tensor matching model input shape (1, 1, 28, 28) for FashionMNIST
batch_size = 1
dummy_input = np.random.randn(batch_size, 1, 28, 28).astype(np.float32)
print(f"Dummy input shape: {dummy_input.shape}")

# Prepare input batch
input_batch = {input_names[0]: dummy_input}

# Run inference
outputs = ort_session.run(output_names, input_batch)
print("\nInference successful!")
print(f"Output shape: {outputs[0].shape}")
print(f"Output (logits) first 5 values: {outputs[0][0][:5]}")

# Get predicted class
predicted_class = np.argmax(outputs[0])
print(f"Predicted class: {predicted_class}")

Dummy input shape: (1, 1, 28, 28)

Inference successful!
Output shape: (1, 10)
Output (logits) first 5 values: [ -8.926842    1.7564752 -16.058088   -2.9262023 -16.068573 ]
Predicted class: 5
